# Phenotype classifcation using CellX 

This notebook shows how to take segmented time lapse microscopy images and use h2b fluorescence markers to classfiy mitotic state of the cell cycle. 

The sections of this notebook are as follows:

1. Load images
2. something
3. something else
4. Class

The data used in this notebook is timelapse microscopy data with h2b-gfp/rfp markers that show the spatial extent of the nucleus and it's mitotic state. 

This notebook uses the dask octopuslite image loader from the CellX/Lowe lab project.

In [2]:
from octopuslite import DaskOctopusLiteLoader

## 1. Load segmentation images

In [10]:
# load images
expt = 'MK0003'
pos = 'Pos15'
image_path = f'/home/nathan/data/kraken/test/{expt}/{pos}/{pos}_stardist_masks_dasktest'
images = DaskOctopusLiteLoader(image_path, crop=(1200,1600))

Using cropping: (1200, 1600)


# Save result of segmentation out as channel099?

In [11]:
images.channels

[<Channels.BRIGHTFIELD: 0>, <Channels.GFP: 1>, <Channels.MASK: 99>]

In [7]:
images.files('GFP')

['/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img_channel001_position015_time000000000_z000_stardist_masks.tif',
 '/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img_channel001_position015_time000000001_z000_stardist_masks.tif',
 '/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img_channel001_position015_time000000002_z000_stardist_masks.tif',
 '/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img_channel001_position015_time000000003_z000_stardist_masks.tif',
 '/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img_channel001_position015_time000000004_z000_stardist_masks.tif',
 '/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img_channel001_position015_time000000005_z000_stardist_masks.tif',
 '/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img_channel001_position015_time000000006_z000_stardist_masks.tif',
 '/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks/img

In [ ]:
gfp = octopus["GFP"]
rfp = octopus["RFP"]

## 2. Localise the objects

In [ ]:
objects_gfp = btrack.utils.segmentation_to_objects(
    segmentation_gfp,
    properties = ('area', ),
)

In [ ]:
objects_rfp = btrack.utils.segmentation_to_objects(
    segmentation_rfp,
    properties = ('area', ),
)

## 3. Classify the objects 

In [ ]:
from cellx import load_model
from cellx.tools.image import InfinitePaddedImage
from skimage.transform import resize

MODEL_PATH = "/media/quantumjot/Data/TrainingData/CNN_training_MDCK_v2_merged"

model = load_model(os.path.join(MODEL_PATH, "cellx_classifier_stardist.h5"))

## 4. Filter the objects 

In [ ]:
filtered_objects = [o for o in objects if o.properties['area']>100.]

## 5. Run btrack  

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(
        "/home/quantumjot/Dropbox/Code/py3/BayesianTracker/models/MDCK_config_scribble_dense.json"
    )
    tracker.max_search_radius = 40

    # append the objects to be tracked
    tracker.append(filtered_objects)

    # set the volume
    tracker.volume=((0, 1600), (0, 1200), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(PATH, 'tracking.h5'), obj_type='obj_type_2')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=2)
    
    tracks = tracker.tracks